In [1]:
from IPython.display import HTML, Javascript
from NeuralNetwork import NeuralNetwork as NN
import json
import threading
import random

In [2]:
nn = NN(2,[4,4],2)
nn.learingRate = 20
answerkey = [True,False]


In [3]:
def nnJson():
    print(nn.toJSON())

In [4]:
def xor(arg1,arg2):
    a = 0
    b = 0
    status = 'unknown'
    if(arg1):
        a= 0.8
    else:
        a = 0.1
    if(arg2):
        b = 0.8
    else:
        b = 0.1
    result = answerkey[nn.predict([a,b]).argmax()]
    if result == arg1 ^ arg2:
        status = 'good'
    else:
        status = 'fail'
    return json.dumps({'status':status,'result':'{} xor {} = {}'.format(arg1,arg2,result)})
    
    

In [5]:
def train(cycles):
    cycle = 0
    data = [{'target':[0.1,0.8], 'input':[0.1,0.1]},
        {'target':[0.8,0.1], 'input':[0.1,0.8]},
        {'target':[0.8,0.1], 'input':[0.8,0.1]},
        {'target':[0.1,0.8], 'input':[0.8,0.8]}
       ]
    while cycle < cycles:
        cycle += 1
        random.shuffle(data)
        for d in data:
            nn.train(d['input'],d['target'])
    nn.predict(data[0]['input'])
        

In [6]:
%%html
<h1>Neural Network Trainer</h1>


<div class="input-group">
  <div class="input-group-append">
    <span class="input-group-text">Cycles</span>
  </div>
  <input id="cycles" type="text" class="form-control" aria-label="Cycles" value='100'></input>
  <button id="go" class="btn btn-default">GO!</button>
</div>

<div>
    <div class='container'>
        <div class='row'>
          <div class='col-sm-4' id='weightCol'>
            Weights
          </div>
          <div class='col-sm-4' id='biasCol' style="width:150px;">
            Biases
          </div>
          <div class='col-sm-4' id='layerCol'style="width:150px;">
            Layers
          </div>
        </div>
        
    </div>
    
    
    <div id='errors'></div>
</div>

In [20]:
print(xor(True,True))
print(xor(False,False))
print(xor(True,False))
print(xor(False,True))

{"status": "good", "result": "True xor True = False"}
{"status": "good", "result": "False xor False = False"}
{"status": "good", "result": "True xor False = True"}
{"status": "good", "result": "False xor True = True"}


In [8]:
%%javascript
function nnToTable(data) {
    //console.log(data)
    if(data.content.ename !== undefined ) {
        var content = data.content
        $('#errors').html(`<h1>${content.ename}</h1><h2>${content.evalue}</h2><p>${JSON.stringify(content.traceback)}</p>`);
        return
    } 
    
    var nndata = JSON.parse(data.content.text.trim());
    //console.log(nndata);
    var wtable = buildTableFromMatrix(nndata.weights[0])
    //console.log(`returned table:${wtable}`)
    //console.log(wtable)
     $('#weightCol').empty()
    $('#weightCol').append(`<h3>Weights</h3>`)
    for(var i = 0 ; i < nndata.weights.length; i++) {
        $('#weightCol').append(buildTableFromMatrix(nndata.weights[i]))
    }
    $('#biasCol').empty()
    $('#biasCol').append(`<h3>Biases</h3>`)
    for(var i = 0 ; i < nndata.weights.length; i++) {
        $('#biasCol').append(buildTableFromMatrix(nndata.biases[i]))
    }
    $('#layerCol').empty()
    $('#layerCol').append(`<h3>Layers</h3>`)
    for(var i = 0 ; i < nndata.weights.length; i++) {
        $('#layerCol').append(buildTableFromMatrix(nndata.layers[i]))
    }
    
    $('.row td').each(function() {
        var bg = toGreyscale(parseFloat($(this).text()));
        //console.log(gscale)
        var fg = 255-bg
        $(this).css('background-color',`rgb(${bg},${bg},${bg})`);
        $(this).css('color',`rgb(${fg},${fg},${fg})` )
    });
}


function buildTableFromMatrix(matrix) {
    //console.log(`Building Table from ${matrix.length} x ${matrix[0].length} matrix`)
    var table = $('<table>');
    table.addClass('table table-striped table-bordered table-dark table-sm')
    for(var row = 0; row < matrix.length; row++) {
        var tr = $('<tr>');
        for(var col = 0; col < matrix[row].length; col++ ) {
            tr.append(`<td>${matrix[row][col].toFixed(5)}</td>`)
        } table.append(tr)
    }
    
    return table
}

function map (num, in_min, in_max, out_min, out_max) {
  return (num - in_min) * (out_max - out_min) / (in_max - in_min) + out_min;
}

function toGreyscale(num) {
    return map(num, 0.0,1.0,0,255)
}

var timer = null;
$('#go').click(function() {
    $('#go').attr('disabled','disabled');
     var cycles = 10
    if($.isNumeric($("#cycles").val())) {
      cycles = parseInt($("#cycles").val())
    }
    var kernel = IPython.notebook.kernel;
    var nnCallbacks = {
        iopub : {
            output: nnToTable,
            }
        };
    //kernel.execute('threading.thread(target=train, args=(10000,)).start()');
    var timerCount = 0
    timer = setInterval(function() {
        timerCount++
        //console.log(timerCount)
        $("#cycles").val(cycles-timerCount)
        kernel.execute('train(100)')
        if(timerCount > cycles) {
            $('#go').removeAttr('disabled');
            clearInterval(timer)
            console.log('suspending Timer')
        }
        kernel.execute('nnJson()',nnCallbacks)},100)
    
});

<IPython.core.display.Javascript object>

In [9]:
nn.learingRate

20

In [16]:
xor(False,False)

'{"status": "good", "result": "False xor False = False"}'

In [17]:
xor(True, True)

'{"status": "good", "result": "True xor True = False"}'

In [18]:
xor(True, False)

'{"status": "good", "result": "True xor False = True"}'

In [19]:
xor(False, True)

'{"status": "good", "result": "False xor True = True"}'